In [ ]:
import module_loader
import pandas as pd
from bookirds.curves import *
from bookirds.dual import Dual

In [ ]:
nodes = {
    datetime(2022, 1, 1): Dual(1, {"v0": 1}),
    datetime(2024, 1, 1): Dual(1, {"v1": 1}),
    datetime(2027, 1, 1): Dual(1, {"v2": 1}),
    datetime(2032, 1, 1): Dual(1, {"v3": 1}),
    datetime(2052, 1, 1): Dual(1, {"v4": 1}),
    datetime(2072, 1, 1): Dual(1, {"v5": 1}),
}
swaps = {
    Swap2(datetime(2022, 1, 1), 12*2, 12, 12): -0.21,
    Swap2(datetime(2024, 1, 1), 12*3, 12, 12): -0.07,
    Swap2(datetime(2027, 1, 1), 12*5, 12, 12): 0.76,
    Swap2(datetime(2032, 1, 1), 12*20, 12, 12): 1.03,
    Swap2(datetime(2052, 1, 1), 12*20, 12, 12): 0.59,
}
labels = ["2Y", "2Y3Y", "5Y5Y", "10Y20Y", "30Y20Y"]
s_cv = SolvedCurve(nodes=nodes, interpolation="log_linear", swaps=list(swaps.keys()), obj_rates=list(swaps.values()))
s_cv.iterate() 

corr = np.array([
    [1, 0.85, 0.81, 0.75, 0.71],
    [0.85, 1, 0.87, 0.81, 0.81],
    [0.81, 0.87, 1, 0.92, 0.89],
    [0.75, 0.81, 0.92, 1, 0.96],
    [0.71, 0.81, 0.89, 0.96, 1],
])
vol = np.array([15.1, 36.6, 58.6, 68.8, 67.3])
Q = np.matmul(np.matmul(np.diag(vol), corr), np.diag(vol))
mu = np.array([[0, -9.3, -20.4, 1.7, 1.4]]).T 

# First Order Roll/Vol Matrix

In [ ]:
sharpe = np.empty((5,5))
sharpe[:] = np.nan
for i in range(4):
    swap = list(swaps.keys())[i]
    a = swap.analytic_delta(s_cv).real
    swap.notional *= -10000 / a
    
    for j in range(i + 1, 5):
        swap2 = list(swaps.keys())[j]
        a = swap2.analytic_delta(s_cv).real
        swap2.notional *= 10000 / a
        
        portfolio = Portfolio([swap, swap2])
        sharpe[j,i] = portfolio.sharpe(s_cv, mu, Q, order=1)
        sharpe[i,j] = -sharpe[j,i]
    
df1 = pd.DataFrame(sharpe[1:, 1:], index=labels[1:], columns=labels[1:])
df1

# Second Order Roll/Vol Matrix

In [ ]:
sharpe = np.empty((5,5))
sharpe[:] = np.nan
for i in range(4):
    swap = list(swaps.keys())[i]
    a = swap.analytic_delta(s_cv).real
    swap.notional *= -10000 / a
    
    for j in range(i + 1, 5):
        swap2 = list(swaps.keys())[j]
        a = swap2.analytic_delta(s_cv).real
        swap2.notional *= 10000 / a
        
        portfolio = Portfolio([swap, swap2])
        sharpe[j,i] = portfolio.sharpe(s_cv, mu, Q, order=2)
        sharpe[i,j] = -sharpe[j,i]
    
df2 = pd.DataFrame(sharpe[1:, 1:], index=labels[1:], columns=labels[1:])
df2

# Effect of Gamma on Roll/Vol Matrix

In [ ]:
df2 - df1

# Efficient Frontier and Trade Sampling

In [ ]:
N = 40000
sample_risk = (np.random.rand(5, N) - 0.5)*2000
sample_risk[0, :] = 0
sample_risk[:, 0:3]  # display the first 3 samples

Use the simple formula to estimate G

In [ ]:
estimator = np.diag([3, 8, 16, 41, 81]) * -1/10000
_ = np.einsum('rn, rj -> rjn', sample_risk, np.eye(5))
sample_gamma = np.einsum('ijn, ij -> ijn', _, estimator)
sample_gamma[:,:,0]  # display estmated gamma for the first sample

In [ ]:
pf = Portfolio()

In [ ]:
pnl1 = [pf.exp_pnl(None, mu, Q, order=1, S=sample_risk[:,i], G=sample_gamma[:,:,i]) for i in range(N)]
var1 = [pf.var_pnl(None, mu, Q, order=1, S=sample_risk[:,i], G=sample_gamma[:,:,i]) for i in range(N)]
vol1 = [v ** 0.5 for v in var1]
pnl2 = [pf.exp_pnl(None, mu, Q, order=2, S=sample_risk[:,i], G=sample_gamma[:,:,i]) for i in range(N)]
var2 = [pf.var_pnl(None, mu, Q, order=2, S=sample_risk[:,i][:, np.newaxis], G=sample_gamma[:,:,i]) for i in range(N)]
vol2 = [v ** 0.5 for v in var2]

baseline_risk = np.array([[0,1,0,0,0], [0,0,1,0,0], [0,0,0,1,0], [0,0,0,0,1]]).T * -1000
_ = np.einsum('rn, rj -> rjn', baseline_risk, np.eye(5))
baseline_gamma = np.einsum('ijn, ij -> ijn', _, estimator)
pnl_base = [pf.exp_pnl(None, mu, Q, order=2, S=baseline_risk[:,i][:, np.newaxis], G=baseline_gamma[:,:,i]) for i in range(4)]
var_base = [pf.var_pnl(None, mu, Q, order=2, S=baseline_risk[:,i][:, np.newaxis], G=baseline_gamma[:,:,i]) for i in range(4)]
vol_base = [v ** 0.5 for v in var_base]

In [ ]:
sharpe = np.array([pnl2[i] / vol2[i] for i in range(N)])
best = np.argmax(sharpe)

import matplotlib.pyplot as plt
fig, ax = plt.subplots(1,1)
# ax.scatter(vol1, pnl1, s=0.2, c="g")
ax.scatter(vol2, pnl2, s=0.2)
ax.scatter(vol_base, pnl_base, s=100.0, marker="s")
ax.scatter(vol2[best], pnl2[best], s=200, marker="*", c="r")
ax.set_xlim(0,1e5)
ax.set_ylim(-35000,35000)
plt.show()

Best risk position assuming roll down as the expected market move is,

In [ ]:
pd.DataFrame(sample_risk[:, best], index=labels).style.format("{:,.0f}").applymap(lambda v: "color: red" if v < 0 else "")

The Sharpe ratio of this trade is:

In [ ]:
pf.sharpe(None, mu, Q, order=1, S=sample_risk[:,best][:, np.newaxis], G= sample_gamma[:,:,best])